In [205]:
import json
import pandas as pd
import numpy as np

import requests
from flask_restx import Model
from pydantic import BaseModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.datasets import make_classification
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

Доступные ручки:

In [206]:
info_endpoint = 'http://127.0.0.1:5000/test_api/ml_models/task_result?task_id={0}'
train_endpoint = 'http://127.0.0.1:5000/test_api/ml_models/train'
delete_endpoint = 'http://127.0.0.1:5000/test_api/ml_models/delete'
predict_endpoint = 'http://127.0.0.1:5000/test_api/ml_models/predict'

Посмотрим, как работает созданное API. Для этого запустим файл `my_api.py` и покидаем в ручки запросы. Первая ручка `info_endpoint` - с информацией об обученных моделях в базе данных `MongoDB`

In [208]:
r = requests.get(train_endpoint).json()
print(requests.get(info_endpoint.format(r)).json())

Создадим пару тестовых датасетов

In [215]:
x_1, y_1 = make_classification(n_samples=400, n_features=5, n_classes=2)

dataset_train_1 = pd.DataFrame(x_1[:100, :])
dataset_train_1['target'] = y_1[:100]
dataset_train_1 = dataset_train_1.to_json()

dataset_train_2 = pd.DataFrame(x_1[100:300, :])
dataset_train_2['target'] = y_1[100:300]
dataset_train_2 = dataset_train_2.to_json()

dataset_test = pd.DataFrame(x_1[300:, :])
dataset_test = dataset_test.to_json()

Отправим запрос с обучением случайного леса с заданными гиперпараметрами; в ответ получаем небольшой отчет об основных метриках обучения на 5 фолдах 

In [216]:
json_train_data = {
    'model_name': 'rfc',
    'model_type': 'Random Forest',
    'hyperparams': {'n_estimators': 200, 'max_depth': 10},
    'train_data': dataset_train_1,
}

r = requests.post(train_endpoint, json=json_train_data)
print(r.json()['report'])

15c79d19-178f-4e7f-8493-2efc94b430de


In [217]:
requests.get(info_endpoint.format(r.json()['report'])).json()

'{"score":{"precision":0.883,"recall":0.807,"roc_auc":0.913,"f1_score":0.839}}'

Проверим, что в списке появилась обученная модель

In [212]:
r = requests.get(train_endpoint).json()
print(requests.get(info_endpoint.format(r)).json())

rfc



Отправим теперь `dataset_train_2`, на котором переобучим созданную `rfc` модель

In [213]:
json_train_data = {
    'model_name': 'rfc',
    'new_data': dataset_train_2,
}

r = requests.put(train_endpoint, data=json_train_data)
print(r.json()['report'])

c5db05b8-70f1-45d5-bd2b-f72adb1b45a0


In [214]:
requests.get(info_endpoint.format(r.json()['report'])).json()

'{"score":{"precision":0.935,"recall":0.961,"roc_auc":0.976,"f1_score":0.947}}'

Проверим, что обученная модель работает - дадим ей на вход оставшуюся часть датасета и посмотрим на ответы

In [179]:
json_pred_data = {
    'model_name': 'rfc',
    'predict_data': dataset_test,
}

r = requests.post(predict_endpoint, json=json_pred_data)
print(r.json()['answers'])


6c1656aa-5eea-4aee-874e-3a905a2fd03a


In [180]:
requests.get(info_endpoint.format(r.json()['answers'])).json()

[0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1]

Напоследок удалим созданную модель с помощью ручки `delete_endpoint` и выведем список обученных моделей

In [181]:
r = requests.delete(delete_endpoint + '/' + 'rfc')
r.json()['result']

'7220cb17-e34a-438c-9c14-5993994124df'

In [182]:
r = requests.get(train_endpoint).json()
print(requests.get(info_endpoint.format(r)).json())

Сохраненных моделей в базе данных не осталось